# Домашнее задание «Продвинутый pandas 2»

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [12]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [61]:
users_100 = ratings.groupby('userId').agg(['min', 'max', 'count'])['timestamp'].query('count > 100')
users_100

,min,max,count
userId,,,
4,949778714,949982274,204
8,1154389340,1154474527,116
15,997937239,1469330735,1700
17,1127468587,1127476640,363
19,855190091,855195373,423
...,...,...,...
656,986240991,986244044,128
659,834598040,866207451,142
664,1343731283,1441911722,519


In [62]:
print(f"Среднее время жизни пользователей, которые выставили более 100 оценок равно: {users_100['max'].mean() - users_100['min'].mean()} секунд")

Среднее время жизни пользователей, которые выставили более 100 оценок равно: 40080507.44961238 секунд


## Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов


In [51]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd


,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [63]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [64]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [65]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [77]:
rzd_auto_air = rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
rzd_auto_air

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [79]:
rzd_auto_air_address = rzd_auto_air.merge(client_base, how='outer', on='client_id')
rzd_auto_air_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

### Ответ:
- нужно добавить время покупки и время, когда были зафиксированы координаты. Вероятно это поможет нам понять где человек находился в момент покупки. Сравнивая время покупки и координаты в этот момент мы можем понять, откуда пользователь делал заказ. Если дневное время - вероятнее всего с работы, если вечернее - из дома. 
- Никак. Если нас интересует только координаты в момент покупки, то мы сравниваем время покупки и какие координаты были в это время. Если я правильно понял вопрос.
- Из набора координат можно сделать ряд предполложений по отношению к данному пользователю: если много удаленных друг от дргуа точек, то вероятно пользовательно проводит много времени в дороге; если наборот, постоянно одни и те же координаты, то можно предположить, чот пользователь все время проводит дома; если доминируют жве локации, то веротяно это дом и место работы пользователя. Эту информацию можно использовать для персональных рекомендация пользователю.

P.S. Данное задание довольно сложно для восприятия. Трудно понять что именно требуется. Я написал ответ, но вообше не уверен это ли от меня хотели.